In [1]:
import pandas as pd                     #导包
import numpy as np
import re
res=pd.read_csv('res.csv')              #读取属性数据集
df=pd.read_csv('df.csv')                #读取规则数据集
df_A = df[(df['target']=='A')]          #读取属性数据集中target=A的数据集
len_A = len(df_A)                       #计算target=A的数据集的数量
len_B = 100000-len_A                    #计算target=B的数据集的数量

In [17]:
###################数据预处理###########################
rules = res.ix[:,1]                                     #提取规则数据集中的规则列
new_rules = []                                          #创建一个list存放计算过后的规则
reExp1 = r'X(.*?)=(.*?)[,}]'                            #正则表达式过滤条件(提取等号两边的数值，组成一个tuple)
for tup in rules:                                       #遍历所有规则------------>复杂度O(R)
    invert = []                                         #创建一个list存放正则后的结果
    invert = re.compile(reExp1).findall(tup)
    tmp = map(lambda x:int(x[0]) + int(x[1]),invert)    #将每个tuple中的数值相加
    tmp = list(tmp)                                     #将结果转成list类型
    tmp.sort()                                          #将list中数值从小到大排序
    tmp = "".join([str(i) for i in tmp])                #将数值合并成一个数字，(例如[2,8,10]-->[2810])
    new_rules.append(tmp)                               #将结果添加到new_rules
dic = dict(zip((res.ix[:,0]),new_rules))                #新建一个字典，key=规则序号，values=新的rules
####################按照相互等价的为一类分类############
rev_dic = {}                                            #新建一个字典存放等价规则分类后的规则，key=rules，values=具有相同rules的rules序号                                     
for k,v in dic.items():                                 #遍历dic---------------->复杂度O(R)
    rev_dic.setdefault(v, []).append(k)                 #将相同规则的序号放到一起--->复杂度O(1)

In [18]:
len(rev_dic)

1495

In [ ]:
d={'819': [572, 581, 1593, 1601, 2084, 2085, 2097, 2346, 2347, 2359],
 '2519': [31434, 31443, 48261, 48270, 51163, 51408],
 '31214': [21648, 21651, 21652, 21669, 21674, 22189, 22277, 22302]}
d

In [ ]:
rul = []                                                           #新建list存放规则
interest =[]                                                       #新建list存放每个类最大的interest值
for k,v in rev_dic.items():                                        #
    NN = []                                                        #新建list存放每个key中的所有interest值
    for j in v:                                                    #循环每个key对应的values--->前两个循环O(R)
        ex = rules[j-1]
        invert = []
        invert = re.compile(reExp1).findall(ex)
        tmpe = list(map(lambda x :(int(x[0]),int(x[1])),invert))
        #tmpe = list(tmpe)
        TL = 74823*[True]
        for sing_tmpe in tmpe:                                     #循环每个value对应的规则的每一个项---->O(K)
            b = df_A.ix[:,sing_tmpe[0]] == sing_tmpe[1]
            TL = TL & b
            A = sum(TL)
        inter = (100000*A - A*A)/(len_A*len_B)
        NN.append(inter)
    N = max(NN)
    interest.append(N)
    rul.append(rules[v[NN.index(max(NN))]])
pd.DataFrame({'interest':interest,'rules':rul},columns=['rules','interest'])

In [ ]:
rev_dict